<a href="https://colab.research.google.com/github/ThaminduSulakshana/CustomChatbot-Tutorial/blob/main/ChatGPT_without_OpenAI_APIs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain
!pip install PyPDF2 langchain
!pip install sentence-transformers
!pip install faiss-gpu

# Convert a document into chunks of text using langchain text splitter

In [ ]:
import PyPDF2
from langchain.text_splitter import SpacyTextSplitter
from sentence_transformers import SentenceTransformer
import pandas as pd
import faiss

In [ ]:
# Define a function to extract text from a PDF file
def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        num_pages = len(pdf_reader.pages)
        for page_number in range(num_pages):
            page = pdf_reader.pages[page_number]
            text += page.extract_text()
    return text

pdf_path = '/content/drive/MyDrive/bluechiptech.asia/LangChain-Chat-bot/docs/MachineLearning-Lecture01.pdf'

In [ ]:
# Extract text from the PDF
pdf_text = extract_text_from_pdf(pdf_path)

# Create a SpacyTextSplitter instance with a specified chunk size
text_splitter = SpacyTextSplitter(chunk_size=500)

# Split the text into chunks
texts = text_splitter.split_text(pdf_text)

/usr/local/lib/python3.10/dist-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


In [ ]:
def extract_first_line_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        num_pages = len(pdf_reader.pages)

        if num_pages > 0:
            first_page = pdf_reader.pages[0]
            first_line = first_page.extract_text().split('\n')[0]
            return first_line
        else:
            return "No pages found in the PDF."

In [ ]:
# Extract and print the first line from the PDF
first_line = extract_first_line_from_pdf(pdf_path)
print("First line from the PDF:", first_line)

First line from the PDF: MachineLearning-Lecture01  


In [ ]:
def extract_first_100_words_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        num_pages = len(pdf_reader.pages)

        if num_pages > 0:
            first_page = pdf_reader.pages[0]
            text = first_page.extract_text()
            words = text.split()[:100]
            first_100_words = ' '.join(words)
            return first_100_words
        else:
            return "No pages found in the PDF."

In [ ]:
# Extract the first 100 words from the PDF
first_100_words = extract_first_100_words_from_pdf(pdf_path)
print(first_100_words)

MachineLearning-Lecture01 Instructor (Andrew Ng): Okay. Good morning. Welcome to CS229, the machine learning class. So what I wanna do today is ju st spend a little time going over the logistics of the class, and then we'll start to talk a bit about machine learning. By way of introduction, my name's Andrew Ng and I'll be instru ctor for this class. And so I personally work in machine learning, and I' ve worked on it for about 15 years now, and I actually think that machine learning is th e most exciting field of all the computer sciences. So I'm


In [ ]:
# Encode the text using SentenceTransformer
encoder = SentenceTransformer("paraphrase-mpnet-base-v2")
vectors = encoder.encode(texts)

In [ ]:
# Build the FAISS index
index = faiss.IndexFlatIP(vectors.shape[1])
index.add(vectors)

In [ ]:
df = pd.DataFrame({'facts': texts})
question_counter = 0

In [ ]:
df.head()

,facts
0,MachineLearning-Lecture01 \nInstructor (Andre...
1,"By way of introduction, my name's Andrew Ng\n..."
2,So I'm actually always excited about teaching...
3,Paul Baumstarck \nworks in machine learning an...
4,Zico Kolter is the head TA — he's head TA two...


In [ ]:
# Define the search function
def search(search_text, encoder, index, df):
    # Encode the search text
    search_vector = encoder.encode([search_text])

    # Normalize the search vector
    faiss.normalize_L2(search_vector)

    # Perform similarity search
    k = index.ntotal
    distances, ann = index.search(search_vector, k=k)
    results = pd.DataFrame({'distances': distances[0], 'ann': ann[0]})

    # Merge with the original dataframe
    merge = pd.merge(results, df, left_on='ann', right_index=True)

    return merge

In [ ]:
# Example search query
search_text = 'Why does Andrew Ng believe in machine learning'

# Perform the search
result = search(search_text, encoder, index, df)

In [ ]:
# Print the results
print(result)

     distances  ann                                              facts
0     1.658532   97  Here's a more recent, a more modern, more form...
1     1.488541   24  And of course, learning algorithms are also  d...
2     1.481760   14  So I have more logistics to go over later, but...
3     1.468557    0  MachineLearning-Lecture01  \nInstructor (Andre...
4     1.465731  129  Okay?  \n\n\nSo I think learning algorithms ar...
..         ...  ...                                                ...
182   0.231938  144  And so the images you see on the bottom — I gu...
183   0.230419  158  So I'm gonna play some audi o clips that were ...
184   0.230002  156  But what we're gonna do is we'll put two \nmic...
185   0.226047   89  I see.\n\nOkay, cool.\n\nOkay.\n\nNo, that's f...
186   0.190356  155  So imagine a large cocktail party with lots of...

[187 rows x 3 columns]


In [ ]:
# Extract the nearest result
nearest_result = result.iloc[0]

# Extract information from the nearest result
nearest_distance = nearest_result['distances']
nearest_ann = nearest_result['ann']
nearest_text = df.loc[nearest_ann, 'facts']

# Display the information
print(f"Nearest Distance: {nearest_distance}")
print(f"Nearest ANN: {nearest_ann}")
print(f"Nearest Text:\n{nearest_text}")


Nearest Distance: 1.6585322618484497
Nearest ANN: 97
Nearest Text:
Here's a more recent, a more modern, more formal definition of machine learning due to 
Tom Mitchell, who says that a well-posed le arning problem is defined as follows: He 
says that a computer program is set to lear n from an experience E with respect to some 
task T and some performance measure P if  its performance on T as measured by P 
improves with

experience E. Okay.

So not  only is it a definition, it even rhymes.


In [ ]:
nearest_index = result['ann'].iloc[0]

# Retrieve the corresponding information from the original dataframe
nearest_match = df.loc[nearest_index, 'facts']

# Display the result
print(f"Question: {search_text}\n")
print(f"Answer: {nearest_match}\n")
print(f"Distance: {result['distances'].iloc[0]}")


Question: Why does Andrew Ng believe in machine learning

Answer: Here's a more recent, a more modern, more formal definition of machine learning due to 
Tom Mitchell, who says that a well-posed le arning problem is defined as follows: He 
says that a computer program is set to lear n from an experience E with respect to some 
task T and some performance measure P if  its performance on T as measured by P 
improves with

experience E. Okay.

So not  only is it a definition, it even rhymes.

Distance: 1.6585322618484497


In [ ]:
# Interactive loop
while True:
    # Increment question counter
    question_counter += 1

    # Input question
    search_text = input(f"Type your question (type 'exit' to quit): ")

    # Check for exit condition
    if search_text.lower() == 'x':
        break

    # Perform the search
    result = search(search_text, encoder, index, df)

    # Display the results
    if not result.empty:
        nearest_index = result['ann'].iloc[0]
        nearest_match = df.loc[nearest_index, 'facts']
        print(f"\n{question_counter} Question: {search_text}\n")
        print(f"Nearest Match: {nearest_match}\n")
        print(f"Distance: {result['distances'].iloc[0]}\n")
    else:
        print("No matching result found.\n")

Type your question (type 'exit' to quit): What are the three goals that are mentioned in machine learning?

1 Question: What are the three goals that are mentioned in machine learning?

Nearest Match: So in teaching this class, I sort of have thre e goals.

One of them is just to I hope convey 
some of my own excitement a bout machine learning to you.  


The second goal is by the end of this class, I hope all of you will be ab le to apply state-of-
the-art machine learning algorithms to whatev er problems you're interested in.

And if you 
ever need to build a system for reading zi p codes, you'll know how to do that by the end 
of this class.

Distance: 1.8445061445236206

Type your question (type 'exit' to quit): Machine learning based on the backgrounds of the TAs

2 Question: Machine learning based on the backgrounds of the TAs

Nearest Match: And 
Daniel Ramage is — I guess he's not here  — Daniel applies l earning algorithms to 
problems in natural language processing.  


So yo